# Open and read CSV

In [1]:
from string import punctuation
import re
import itertools
import pickle
import time

In [2]:
import os
cwd = os.getcwd()
print(cwd)

C:\Users\Benjamin\Dropbox\DH\Eigene\ZeitromanGBVplusDLA\Notebooks


In [3]:
import csv

counter = 0
books = {}


### READING prefiltered DLA&DNB-data (all appearences of "zeitroman" / "zeit-roman" )
with open(r"ZeitromanDLAplusGBV.csv") as f:
    rd = csv.DictReader(f, delimiter=';')
    
    for row in rd:
        counter += 1
        #key = row[0]
        key = row.pop('risn', None) # WICHTIG: hier Kopf der Nummerierung einfügen
        if key in books:
            print("key ist doppelt")
            break
            pass
        if counter > 1000:
            break
            pass
        books[key] = row
#print(books)

count_fictions = len(books)
print('Anzahl der Titel:', count_fictions)

Anzahl der Titel: 442


# Get `author`, `title`, `date` for leftover books.

In [4]:
def get_info(b):
    
    title_id = b
    try:
        author = books[b]['XX_PE_listview_title_PE0100']
        #author = author.split('(', 1)[0]
        #author = author.split(';', 1)[0]

    except:
        author = None
        pass
        
    title = books[b]['TITREG']
    
    try:
        sub_title = books[b]['A0335']
    except:
        pass

    try:
        title_without_sub = books[b]['A0331'] #funzt nur bei DLA-Daten
    except:
        title_without_sub = title.rsplit('---', 1)[0]
        pass
    
    
    try:
        date = books[b]['AA425']
        if len(date) == 0:
            date = books[b]['A0425']
            #print('USED A0425')
        if len(date) == 0:
            date = books[b]['A0427']
            dates_list = re.findall("[0-9]{4,}", date)
            dates_list = [int(x) for x in dates_list]
            dates_list.sort
            date = str(dates_list[0])
            #print('USED A0427')
        date = ''.join([c for c in date if c.isnumeric()])
        if len(date) > 4:
            date = date[:4]
        date = int(date)
        
    except:
        date = None
        pass
            
    
    try:
        edition = books[b]['A0403']
    except:
        edition = None
        pass
    
    try:
        location = str.lower(str(books[b]['A0410']))
    except:
        location = None
        pass
    
    try:
        publishing_house = str.lower(str(books[b]['A0412']))
    except:
        publishing_house = None
        pass
        
    try:
        pages = str.lower(str(books[b]['A0433']))
        number_of_pages = pages.split('s', 1)[0]    
       
    except:
        number_of_pages = None
        pass
    
    try:
        genre_by_librarian = books[b]['A0712']
    except:
        genre_by_librarian = None
        pass

    if sub_title:
        return {'author': author,
                'title': title,
                'date': date,
                'subtitle': sub_title,
                'title_without_sub': title_without_sub,
                'edition': edition,
                'location': location,
                'publishing-house': publishing_house,
                'pages': number_of_pages,
                'genre-by-librarian': genre_by_librarian,
                'title_id': title_id}
    else:
        return {'author': author,
                'title': title,
                'date': date,
                'title_without_sub': title_without_sub,
                'edition': edition,
                'location': location,
                'publishing-house': publishing_house,
                'pages': number_of_pages,
                'genre-by-librarian': genre_by_librarian,
                'title_id': title_id}

# Remove everything that isn't first edition:

In [5]:
import re
sifted_books = {} 
for b in books:
        
    try:
        desc = books[b]['A0403']
        desc = desc.replace('[', '').replace(']', '')
        
    except:
        sifted_books[b] = books[b]
        continue

    if desc[0:2] == '1.':
        sifted_books[b] = books[b]
        continue
    else:
        pass
        
    pattern = re.compile(r'([Nn]ova|[Nn]ouv|[Bb]earb|Gek|[Ee]rw|[Ff]aks|[Ss]chul|[Mm]onumental|[Ff]eldpost|[Bb]linden|[Rr]epr|[Nn]eu|[Vv]olksausgabe|[Ll]imited|\[.+\]|\[.+\]|[Aa]ktualisiert|[Nn]achdr|[Kk]ult|[Ee]xclusiv|[Kk]orrigiert|[Ss]tudien|[Dd]urchges)')
    pattern2 = re.compile(r'([023456789]{1,2}\.\s)')
    pattern_non = re.compile(r'([Tt]sd|[Tt]aus\.|[Tt]ausend|[Ee]rstausg|[1]\.\s[Aa]usg|[1]\.\s[Aa]ufl|[1]\.\s[Aa]usg)|[Dd]er\sendg')
    
    if len(re.findall(pattern, desc)) > 0:
        #print(desc)
        continue
        
    if len(re.findall(pattern2, desc)) > 0 and len(re.findall(pattern_non, desc)) == 0:
        #print(desc)
        continue

    try:
        num = int(desc.strip()[0])
        if num > 1 and len(re.findall(pattern_non, desc)) == 0:
            #print(desc)
            continue
        try:
            num1 = int(desc.strip()[1])
            if num1 > 1 and len(re.findall(pattern_non, desc)) == 0:
                #print('Num1 =', num1)
                #print(desc)
                continue
        except:
            pass

    except:
        pass

    sifted_books[b] = books[b]
    #sifted_books.append(b)
#print(dict(list(sifted_books.items())[0:2]))
print('{} of {} German fiction books were first editions'.format(len(sifted_books), count_fictions))

375 of 442 German fiction books were first editions


# Removing Translations

In [6]:
sifted_books2 = {} 
desc = ''

for b in sifted_books:
        
    try:
        desc = sifted_books[b]['A0359'] + ' ' + sifted_books[b]['A0335'] + ' ' + sifted_books[b]['A0501']
        #desc = desc.replace('[', '').replace(']', '')
        
    except:
        sifted_books2[b] = sifted_books[b]
        continue

    pattern245 = re.compile(r'(=|([^\s]+)bers\.|([^\s]+)bersetzt|([^\s]+)bersetzung|([^\s]+)bertr\.|([^\s]+)bertragen|([^\s]+)bertragung)|[Tr]ansl|[Tr]aduz|[Tr]aduc|[Tt]raduit|[Tt]radot|[Tt]rad\.|[Oo]versat|[Pp]ereved|([^\s]+)versatt|[Pp]erekl|[Pp]rzet([^\s]+)umacz|p([^\s]+)elo([^\s]+)eno|[Pp]reved|[Pp]rze([^\s]+).|[Pp]rze([^\s]+)o([^\s]+)y([^\s]+)|[Oo]versat|[Kk]([^\s]+)([^\s]+)nnet|[Pp]rev|[Ll]eford([^\s]+)tott|[Pp]erev|[Pp]rzek([^\s]+)ad|[Tt]olkning')
    pattern_nicht = re.compile(r'([^\s]+)bersetzungen')
    
    if len(re.findall(pattern245, desc)) > 0 and len(re.findall(pattern_nicht, desc)) == 0:
        #print(desc, '\n')
        continue
        
    else:
        pass

    sifted_books2[b] = sifted_books[b]

#print(dict(list(sifted_books2.items())[0:2]))
print('{} of {} German fiction first editions were not translations'.format(len(sifted_books2), len(sifted_books)))        

375 of 375 German fiction first editions were not translations


# Part 2: Delete duplicates

Function to check whether authors or dates are the same:

In [7]:
def check_authors(a1, a2):

    a1str = a1.strip()
    a2str = a2.strip()
    
    if (len(a1str) == 0 or a1 == None) and (len(a1str) == 0 or a2 == None):
        return True ### wenn beide Titel keine Autoren haben, als Duplikate behandeln?
      
    a1 = ''.join([x for x in a1 if x not in punctuation])
    a2 = ''.join([x for x in a2 if x not in punctuation])
    
    a1_words = a1.split()
    a2_words = a2.split()
    #print("AAA1: ", a1, "AAA2:", a2)
    
    count = 0
    for w in a2_words:
        if w in a1_words:
            count += 1
            
    if count >= 2:
        return True
    else:
        return False
    
def check_dates(d1, d2):
    if d1 == None and d2 != None:
        return 'check'
    elif d2 == None and d1 != None:
        return 'current'
    elif d1 == None and d2 == None:
        return 'check'
    elif d2 > d1:
        return 'current'
    else:
        return 'check'

Remove if author and title are the same, keep earlier date:

In [8]:
no_duplicates = {}

to_del = []
current_title_without_sub = ""
check_b_title_without_sub = ""
infos = {}

for b in sifted_books2:
    current_b = get_info(b)
    current_title_without_sub = current_b['title_without_sub']
    no_append = False
    
    for i, n in enumerate(no_duplicates):
        if not (i in infos):
            infos[i] = get_info(n) 

        
        check_b = infos[i] 
        check_b_title_without_sub = check_b['title_without_sub']
        
        if current_title_without_sub == check_b_title_without_sub:
            same_author = check_authors(current_b['author'], check_b['author'])
            if same_author == True:
                date = check_dates(current_b['date'], check_b['date'])
                if date == 'current':
                    no_append = False
                    to_del.append(n)
                else:
                    no_append = True
                    break
                    
    if no_append == False:
        no_duplicates[b] = sifted_books2[b]
        
for i, z in enumerate(list(set(to_del))):
    del no_duplicates[z]
    
print('{} of {} first edition German novels that were no translations are not duplicates'.format(len(no_duplicates), len(sifted_books2)))

168 of 375 first edition German novels that were no translations are not duplicates


In [9]:
print('{} of {} total records (DLA & DNB, fictions) were kept'.format(len(no_duplicates), count_fictions))

168 of 442 total records (DLA & DNB, fictions) were kept


In [10]:
filehandler = open('no_duplicate.pkl', 'wb')
pickle.dump(no_duplicates, filehandler, pickle.HIGHEST_PROTOCOL)

In [11]:
for_csv = [get_info(x) for x in no_duplicates]

import csv

with open('ZeitromanDLAplusGBV_automatisch_gefiltert.csv', 'w', encoding='utf8') as output_file:
    dict_writer = csv.DictWriter(output_file, delimiter=';', lineterminator='\n', fieldnames=['author', 'title', 'subtitle', 'title_without_sub', 'edition', 'genre', 'date', 'location', 'publishing-house', 'pages', 'genre-by-librarian', 'title_id'])
    dict_writer.writeheader()
    dict_writer.writerows(for_csv)

In [96]:
neue_jahre = []

import pandas as pd
pd.set_option('display.max_rows', None)
df = pd.read_csv('ZeitromanDLAplusGBV_automatisch_gefiltert_und_manuell_bereinigt.csv', delimiter=';', lineterminator='\n')
df_counts = df.groupby(['date']).size().reset_index(name='Untertitel.Zeitroman')

for x in range(1800, 2020):
    if x not in df_counts["date"].values:
        #df_new = pd.DataFrame({"date":[x], "counts":[0]}) 
        neue_jahre.append({"date":x, 
                    "Untertitel.Zeitroman":0})
    else:
        pass
df_new = pd.DataFrame(neue_jahre)
df_counts = df_counts.append(df_new, ignore_index = True).sort_values(by='date')
df_counts = df_counts.rename(columns = {'date':'Jahr'}).sort_values(by='Jahr')
print(df_counts)

     Jahr  Untertitel.Zeitroman
72   1800                     0
73   1801                     0
74   1802                     0
75   1803                     0
76   1804                     0
77   1805                     0
78   1806                     0
79   1807                     0
80   1808                     0
81   1809                     0
82   1810                     0
83   1811                     0
84   1812                     0
85   1813                     0
86   1814                     0
87   1815                     0
88   1816                     0
89   1817                     0
90   1818                     0
91   1819                     0
92   1820                     0
93   1821                     0
94   1822                     0
95   1823                     0
96   1824                     0
97   1825                     0
98   1826                     0
99   1827                     0
100  1828                     0
101  1829                     0
102  183

In [98]:
df_counts.reset_index()[['Jahr', 'Untertitel.Zeitroman']].to_csv('Untertitel_Zeitroman_Jahr.csv', sep=';')